### <center> <span style="color:black">Comparing various Boroughs in Manhattan and London based on social venues data from Four Square and using K - Means Clustering </span> <center>

#### <span style="color:gray"> Installing and Importing all libraries </span>

In [1]:
!pip install pandas
# !conda update pandas
import pandas as pd

In [2]:
import numpy as np

In [3]:
print(pd.__version__)

1.2.4


In [4]:
!pip install bs4
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
!pip install geocoder
import geocoder # import geocoder
import numpy as np # library to handle data in a vectorized manner
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
!pip install -U numpy
!pip install -U pandas
!pip install -U scipy
!pip install -U scikit-learn
!pip install -U imbalanced-learn
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### <span style="color:gray"> Get names of London boroughs via webscraping using Beautiful Soup and read it into a pandas Dataframe  </span>

In [5]:
url= 'https://en.wikipedia.org/wiki/List_of_London_boroughs'
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html5lib")
print(soup.title)
table_contents=[]
tables = soup.find('table')
print(len(tables))
table1=soup.find_all('table')[0]
print(table1.prettify())
lon_boroughs_data = pd.DataFrame(columns=["Neighborhood"])

<title>List of London boroughs - Wikipedia</title>
2
<table class="wikitable sortable" style="font-size:100%" width="100%">
 <tbody>
  <tr>
   <th>
    Borough
   </th>
   <th>
    Inner
   </th>
   <th>
    Status
   </th>
   <th>
    Local authority
   </th>
   <th>
    Political control
   </th>
   <th>
    Headquarters
   </th>
   <th>
    Area (sq mi)
   </th>
   <th>
    Population (2019 est)
    <sup class="reference" id="cite_ref-1">
     <a href="#cite_note-1">
      [1]
     </a>
    </sup>
   </th>
   <th>
    Co-ordinates
   </th>
   <th>
    <span style="background:#67BCD3">
     Nr. in map
    </span>
   </th>
  </tr>
  <tr>
   <td>
    <a href="/wiki/London_Borough_of_Barking_and_Dagenham" title="London Borough of Barking and Dagenham">
     Barking and Dagenham
    </a>
    <sup class="reference" id="cite_ref-2">
     <a href="#cite_note-2">
      [note 1]
     </a>
    </sup>
   </td>
   <td>
   </td>
   <td>
   </td>
   <td>
    <a href="/wiki/Barking_and_Dagenham_Lon

In [6]:
for row in table1.tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        lon_borough_name = col[0].text.strip().split('[')[0]
        lon_boroughs_data = lon_boroughs_data.append({"Neighborhood":lon_borough_name}, ignore_index=True)

#Adding City of London separately to the list of borougha as it is not considered a borough and not listed 
lon_boroughs_data = lon_boroughs_data.append({"Neighborhood":"City of London"}, ignore_index=True)
lon_boroughs_data 

,Neighborhood
0,Barking and Dagenham
1,Barnet
2,Bexley
3,Brent
4,Bromley
5,Camden
6,Croydon
7,Ealing
8,Enfield
9,Greenwich


In [7]:
from pandas import DataFrame

#### <span style="color:gray"> Finding Location Coordinates of each Borough using geocoder and displaying on a map using Folium  </span>

In [8]:
df = DataFrame(lon_boroughs_data,columns=['Neighborhood'])

In [9]:
df=pd.DataFrame(lon_boroughs_data)
latitude = []
longitude = []
for index, row in df.iterrows():
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, London,England'.format(row.Neighborhood))
        lat_lng_coords = g.latlng
        latitude.append(lat_lng_coords[0])
        longitude.append(lat_lng_coords[1])
     
df['Latitude'] = latitude
df['Longitude'] = longitude

df

,Neighborhood,Latitude,Longitude
0,Barking and Dagenham,51.560228,0.171961
1,Barnet,51.627300,-0.253760
2,Bexley,51.452078,0.069931
3,Brent,51.609783,-0.194672
4,Bromley,51.601511,-0.066365
5,Camden,51.591180,-0.165040
6,Croydon,51.593470,-0.083380
7,Ealing,51.508383,-0.305200
8,Enfield,51.540024,-0.077502
9,Greenwich,51.477890,-0.013340


In [10]:
lon_boroughs_data=df
g = geocoder.arcgis('London,England')
lat_lng_coords = g.latlng
latitude=lat_lng_coords[0]
longitude=lat_lng_coords[1]
print(longitude,latitude)
# create map of London using latitude and longitude values
map_lon = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lon)  
    
map_lon

-0.12769869299995662 51.507408360000056


#### <span style="color:gray"> Get names of Manhattan neighbourhoods names and coordinates using pre-existing tutorial file read it into a pandas Dataframe  </span>

In [11]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']

# define the dataframe columns, 

# to start off in this dataframe we will gather all the 5 boroughs of New York and all the Neighbourhoods 
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
  

    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

neighborhoods.head()


Data downloaded!


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [12]:
neighborhoods.shape

(306, 4)

In [13]:

# as New York is much more densely populated than London and London has only 33 boroughs while from above we can see that New York has 306 neighbourhoods within its 5 Boroughs 
# so for parity we will just compare the boroughs of London to the Neighbourhoods of Manhattan and drop the remaining
neighborhoods=neighborhoods.drop(neighborhoods[neighborhoods['Borough']=='Bronx'].index)
neighborhoods=neighborhoods.drop(neighborhoods[neighborhoods['Borough']=='Queens'].index)
neighborhoods=neighborhoods.drop(neighborhoods[neighborhoods['Borough']=='Staten Island'].index)
neighborhoods=neighborhoods.drop(neighborhoods[neighborhoods['Borough']=='Brooklyn'].index)


In [14]:
neighborhoods.shape

(40, 4)

#### <span style="color:gray"> Map the nieghbourhoods of Manhattan onto a New York map using Folium  </span>

In [15]:
# lon_boroughs_data=df
g = geocoder.arcgis('New York City, NY')
lat_lng_coords = g.latlng
latitude=lat_lng_coords[0]
longitude=lat_lng_coords[1]
print(longitude,latitude)

-74.00713999999994 40.71455000000003


In [16]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

#### <span style="color:gray"> Now we start creating our request to Four Square API to get data on the venues around each of the boroughs and neighbourhoods identified </span>

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
# adding a column called Borough to London data and populating it with the value London for all rows 
lon_boroughs_data['Borough']='London'

# concatenating the london boroughs and manhattan neighbourhoods into a single dataframe
lon_ny_data=pd.concat([neighborhoods,lon_boroughs_data])

print(lon_ny_data.head(125))

#creating the url for requesting data from Foursquare
CLIENT_ID = 'NABSXF1SXJPOFODQ1JHQTMNLRZHZRWXB5BG3FG54SVG22PJ1' # your Foursquare ID
CLIENT_SECRET = 'N4TFRK3YKSS1YT3XKOENXZYTFMXPOGGW2HEFVMJY10RB2GM4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
 

       Borough             Neighborhood   Latitude  Longitude
6    Manhattan              Marble Hill  40.876551 -73.910660
100  Manhattan                Chinatown  40.715618 -73.994279
101  Manhattan       Washington Heights  40.851903 -73.936900
102  Manhattan                   Inwood  40.867684 -73.921210
103  Manhattan         Hamilton Heights  40.823604 -73.949688
104  Manhattan           Manhattanville  40.816934 -73.957385
105  Manhattan           Central Harlem  40.815976 -73.943211
106  Manhattan              East Harlem  40.792249 -73.944182
107  Manhattan          Upper East Side  40.775639 -73.960508
108  Manhattan                Yorkville  40.775930 -73.947118
109  Manhattan               Lenox Hill  40.768113 -73.958860
110  Manhattan         Roosevelt Island  40.762160 -73.949168
111  Manhattan          Upper West Side  40.787658 -73.977059
112  Manhattan           Lincoln Square  40.773529 -73.985338
113  Manhattan                  Clinton  40.759101 -73.996119
114  Man

#### <span style="color:gray"> Fetching venues data for our combined list of London and Manhattan neighbourhoods from Four Square  </span>

In [19]:
# defining a function to extract venues for all the nieghbourhoods 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

         

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        

        # return only relevant information for each nearby venue
        venues_list.append([(

            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                    'Neighborhood Latitude', 
                    'Neighborhood Longitude', 
                    'Venue', 
                    'Venue Latitude', 
                    'Venue Longitude', 
                    'Venue Category']
  
    return(nearby_venues)


# function definition complete


# calling the function on our combined neighbourhoods list
lon_ny_venues = getNearbyVenues(names=lon_ny_data['Neighborhood'],
                                   latitudes=lon_ny_data['Latitude'],
                                   longitudes=lon_ny_data['Longitude']
                                  )

print(lon_ny_venues.shape)
print(lon_ny_venues.head())
print('There are {} uniques categories.'.format(len(lon_ny_venues['Venue Category'].unique())))

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards
Barking and Dagenham 
Barnet
Bexley
Brent
Bromley
Camden
Croydon
Ealing
Enfield
Greenwich  
Hackney
Hammersmith and Fulham 
Haringey
Harrow
Havering
Hillingdon
Hounslow
Islington
Kensington and Chelsea
Kingston upon Thames
Lambeth
Lewisham
Merton
Newham
Redbridge
Richmond upon Thames
Southwark
Sutton
Tower Hamlets
Waltham Forest
Wandsworth
Westminster
City of London
(4392, 7)
  Neighborhood  Neighborhood Latitude  Neighborhood Longitude          Venue  \
0  Marble Hill           

#### <span style="color:gray"> Cleaning the Data Venue  </span>

In [20]:
#Cleaning the Data : Venue Categories data - replacing the values as a pub in London is the same as a bar in Manhattan, and a Cafe is the same as a Coffee Shop

lon_ny_venues.loc[lon_ny_venues['Venue Category'] == 'Pub', 'Venue Category'] = 'Bar'
lon_ny_venues.loc[lon_ny_venues['Venue Category'] == 'Café', 'Venue Category'] = 'Coffee Shop'
lon_ny_venues.loc[lon_ny_venues['Venue Category'] == 'Cocktail Bar', 'Venue Category'] = 'Bar'
lon_ny_venues.loc[lon_ny_venues['Venue Category'] == 'Whisky Bar', 'Venue Category'] = 'Bar'
lon_ny_venues.loc[lon_ny_venues['Venue Category'] == 'Wine Bar', 'Venue Category'] = 'Bar'
lon_ny_venues.loc[lon_ny_venues['Venue Category'] == 'Beer Bar', 'Venue Category'] = 'Bar'
lon_ny_venues.loc[lon_ny_venues['Venue Category'] == 'Cafeteria', 'Venue Category'] = 'Coffee Shop'

In [21]:
print('There are {} uniques categories.'.format(len(lon_ny_venues['Venue Category'].unique())))

There are 364 uniques categories.


In [22]:
#Cleaning the Data : Checking if there are any neighbourhoods with less than 10 records and dropping them

lon_ny_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Barking and Dagenham,1,1,1,1,1,1
Barnet,1,1,1,1,1,1
Battery Park City,81,81,81,81,81,81
Bexley,4,4,4,4,4,4
Brent,4,4,4,4,4,4
Bromley,10,10,10,10,10,10
Camden,25,25,25,25,25,25
Carnegie Hill,92,92,92,92,92,92
Central Harlem,43,43,43,43,43,43


In [23]:
lon_ny_venues=lon_ny_venues.drop(lon_ny_venues[(lon_ny_venues['Neighborhood']=='Barnet')|(lon_ny_venues['Neighborhood']=='Bexley')|(lon_ny_venues['Neighborhood']=='Barking and Dagenham ')|(lon_ny_venues['Neighborhood']=='Bergen Beach')|(lon_ny_venues['Neighborhood']=='Canarsie')|(lon_ny_venues['Neighborhood']=='Croydon')|(lon_ny_venues['Neighborhood']=='Madison')|(lon_ny_venues['Neighborhood']=='Marine Park')|(lon_ny_venues['Neighborhood']=='Mill Island')|(lon_ny_venues['Neighborhood']=='Paerdegat Basin')|(lon_ny_venues['Neighborhood']=='Newham')|(lon_ny_venues['Neighborhood']=='Westminster')|(lon_ny_venues['Neighborhood']=='Richmond upon Thames')|(lon_ny_venues['Neighborhood']=='Sea Gate')|(lon_ny_venues['Neighborhood']=='Waltham Forest')].index)
lon_ny_venues=lon_ny_venues.drop(lon_ny_venues[(lon_ny_venues['Neighborhood']=='Brent')|(lon_ny_venues['Neighborhood']=='Dyker Heights')].index)

In [24]:
#checking the count of venues per neighbourhood again
lon_ny_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,81,81,81,81,81,81
Bromley,10,10,10,10,10,10
Camden,25,25,25,25,25,25
Carnegie Hill,92,92,92,92,92,92
Central Harlem,43,43,43,43,43,43
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
City of London,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100


#### <span style="color:gray"> Creating the dataset for running K means  </span>

In [25]:
# one hot encoding
lon_ny_onehot = pd.get_dummies(lon_ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
lon_ny_onehot['Neighborhood'] = lon_ny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [lon_ny_onehot.columns[-1]] + list(lon_ny_onehot.columns[:-1])
lon_ny_onehot = lon_ny_onehot[fixed_columns]

lon_ny_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cajun / Creole Restaurant,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Newsagent,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Peruvian Roast Chicken Joint,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Post Office,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road

In [26]:
#group the onehot data by neighbourhood names
lon_ny_grouped = lon_ny_onehot.groupby('Neighborhood').mean().reset_index()
lon_ny_grouped.shape

(64, 362)

In [27]:
lon_ny_grouped.head(5)

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beach Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cajun / Creole Restaurant,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Newsagent,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Peruvian Roast Chicken Joint,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Post Office,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road

In [49]:
# create a function to identify the most commonly occuring venues 
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [50]:
# retrieve top 10 venues for each neighbourhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = lon_ny_grouped['Neighborhood']

for ind in np.arange(lon_ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(lon_ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Clothing Store,Gym,Playground,Memorial Site,Boat or Ferry,Burger Joint,Pizza Place
1,Bromley,Bar,Hostel,Supermarket,Coffee Shop,Train Station,Mediterranean Restaurant,Soccer Stadium,Sporting Goods Shop,Outdoor Sculpture,Organic Grocery
2,Camden,Coffee Shop,Greek Restaurant,Pizza Place,Bus Stop,Grocery Store,Chinese Restaurant,French Restaurant,Health & Beauty Service,Shoe Store,Movie Theater
3,Carnegie Hill,Coffee Shop,Bar,Wine Shop,Yoga Studio,Gym,French Restaurant,Bookstore,Pizza Place,Gym / Fitness Center,Bakery
4,Central Harlem,Bar,Seafood Restaurant,African Restaurant,American Restaurant,Gym / Fitness Center,Chinese Restaurant,Coffee Shop,French Restaurant,Public Art,Bookstore


In [51]:
neighborhoods_venues_sorted.shape

(64, 11)

#### <span style="color:gray"> Running the K means algorithm on the dataset to identify clusters of similar neighbourhoods across London and Manhattan  </span>

In [52]:
# set number of clusters
kclusters = 5

lon_ny_grouped_clustering = lon_ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lon_ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:126] 

array([1, 0, 4, 1, 1, 1, 1, 1, 1, 1, 0, 4, 0, 0, 1, 1, 1, 0, 1, 0, 4, 0,
       1, 0, 0, 1, 1, 1, 4, 1, 0, 1, 4, 1, 4, 1, 1, 1, 4, 4, 1, 1, 1, 1,
       4, 1, 1, 3, 4, 1, 0, 0, 2, 1, 3, 1, 4, 1, 1, 1, 0, 4, 1, 1],
      dtype=int32)

In [53]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

lon_ny_merged = lon_ny_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
lon_ny_merged = lon_ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
# lon_ny_merged = lon_ny_grouped.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
lon_ny_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Manhattan,Marble Hill,40.876551,-73.910660,1.0,Gym,Sandwich Place,Yoga Studio,Tennis Stadium,Kids Store,Ice Cream Shop,Pharmacy,Pizza Place,Donut Shop,Diner
100,Manhattan,Chinatown,40.715618,-73.994279,1.0,Chinese Restaurant,Bar,Bakery,American Restaurant,Hotpot Restaurant,Ice Cream Shop,Dessert Shop,Salon / Barbershop,Optical Shop,Spa
101,Manhattan,Washington Heights,40.851903,-73.936900,4.0,Coffee Shop,Bakery,Grocery Store,Mobile Phone Shop,Gym,Liquor Store,Pizza Place,Sandwich Place,Deli / Bodega,Bar
102,Manhattan,Inwood,40.867684,-73.921210,4.0,Coffee Shop,Mexican Restaurant,Restaurant,Bar,Lounge,Bakery,Pizza Place,Deli / Bodega,Park,Chinese Restaurant
103,Manhattan,Hamilton Heights,40.823604,-73.949688,4.0,Coffee Shop,Pizza Place,Bar,Deli / Bodega,Mexican Restaurant,Bakery,Indian Restaurant,Sandwich Place,Caribbean Restaurant,School


#### <span style="color:gray"> Examining each cluster  </span>

In [54]:
Cluster0=lon_ny_merged.loc[lon_ny_merged['Cluster Labels'] == 0, lon_ny_merged.columns[[0]+[1] + list(range(lon_ny_merged.shape[1]-10, lon_ny_merged.shape[1]))]]


In [55]:
print(Cluster0.shape)
Cluster0


(13, 12)


,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
118,Manhattan,East Village,Bar,Coffee Shop,Mexican Restaurant,Speakeasy,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Italian Restaurant,Pizza Place,Korean Restaurant,Bagel Shop
275,Manhattan,Stuyvesant Town,Bar,Boat or Ferry,Park,Coffee Shop,Harbor / Marina,Gas Station,Heliport,Fountain,Farmers Market,Baseball Field
4,London,Bromley,Bar,Hostel,Supermarket,Coffee Shop,Train Station,Mediterranean Restaurant,Soccer Stadium,Sporting Goods Shop,Outdoor Sculpture,Organic Grocery
7,London,Ealing,Bar,Coffee Shop,Italian Restaurant,Park,Clothing Store,Pizza Place,Bookstore,Performing Arts Venue,Hotel,Pharmacy
8,London,Enfield,Bar,Coffee Shop,Middle Eastern Restaurant,Restaurant,Lounge,Park,Bookstore,Burger Joint,Canal,Gay Bar
9,London,Greenwich,Bar,Coffee Shop,Gym / Fitness Center,Hotel,French Restaurant,Pizza Place,Burger Joint,Dance Studio,Thrift / Vintage Store,Theater
10,London,Hackney,Coffee Shop,Bar,Flower Shop,Art Gallery,Italian Restaurant,Bookstore,Hotel,Yoga Studio,Concert Hall,Beer Store
11,London,Hammersmith and Fulham,Coffee Shop,Park,Bar,Grocery Store,Gym / Fitness Center,Italian Restaurant,Thai Restaurant,Persian Restaurant,Bakery,Pizza Place
13,London,Harrow,Bar,Coffee Shop,Italian Restaurant,Sandwich Place,Hotel,Restaurant,Modern European Restaurant,French Restaurant,History Museum,Asian Restaurant
14,London,Havering,Coffee Shop,Bar,Pharmacy,Vegetarian / Vegan Restaurant,Market,Grocery Store,Park,Ice Cream Shop,Flea Market,Pizza Place


In [56]:
Cluster1=lon_ny_merged.loc[lon_ny_merged['Cluster Labels'] == 1, lon_ny_merged.columns[[0]+[1] + list(range(lon_ny_merged.shape[1]-10, lon_ny_merged.shape[1]))]]

In [57]:
print(Cluster1.shape)
Cluster1

(36, 12)


,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Manhattan,Marble Hill,Gym,Sandwich Place,Yoga Studio,Tennis Stadium,Kids Store,Ice Cream Shop,Pharmacy,Pizza Place,Donut Shop,Diner
100,Manhattan,Chinatown,Chinese Restaurant,Bar,Bakery,American Restaurant,Hotpot Restaurant,Ice Cream Shop,Dessert Shop,Salon / Barbershop,Optical Shop,Spa
105,Manhattan,Central Harlem,Bar,Seafood Restaurant,African Restaurant,American Restaurant,Gym / Fitness Center,Chinese Restaurant,Coffee Shop,French Restaurant,Public Art,Bookstore
107,Manhattan,Upper East Side,Italian Restaurant,Exhibit,Gym / Fitness Center,Bakery,Coffee Shop,American Restaurant,Juice Bar,Hotel,Yoga Studio,Cosmetics Shop
108,Manhattan,Yorkville,Italian Restaurant,Bar,Coffee Shop,Gym,Deli / Bodega,Japanese Restaurant,Wine Shop,Sushi Restaurant,Mexican Restaurant,Diner
109,Manhattan,Lenox Hill,Coffee Shop,Italian Restaurant,Bar,Pizza Place,Sushi Restaurant,Gym / Fitness Center,Gym,Burger Joint,Taco Place,Cycle Studio
111,Manhattan,Upper West Side,Bar,Italian Restaurant,Coffee Shop,Bakery,Mediterranean Restaurant,Vegetarian / Vegan Restaurant,Bookstore,Indian Restaurant,Ice Cream Shop,Sushi Restaurant
112,Manhattan,Lincoln Square,Coffee Shop,Plaza,Concert Hall,Performing Arts Venue,Theater,Park,Wine Shop,French Restaurant,Indie Movie Theater,Bakery
113,Manhattan,Clinton,Italian Restaurant,Theater,Bar,Gym / Fitness Center,American Restaurant,Coffee Shop,Gym,Sandwich Place,Hotel,Spa
114,Manhattan,Midtown,Hotel,Coffee Shop,Sporting Goods Shop,Theater,Clothing Store,Steakhouse,American Restaurant,Bookstore,Japanese Restaurant,Cycle Studio


In [58]:
Cluster2=lon_ny_merged.loc[lon_ny_merged['Cluster Labels'] == 2, lon_ny_merged.columns[[0]+[1] + list(range(lon_ny_merged.shape[1]-10, lon_ny_merged.shape[1]))]]

In [59]:
print(Cluster2.shape)
Cluster2

(1, 12)


,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,London,Sutton,Grocery Store,Park,Pool,Fried Chicken Joint,Coffee Shop,Bar,Gym / Fitness Center,Indian Restaurant,Hotel,Street Art


In [60]:
Cluster3=lon_ny_merged.loc[lon_ny_merged['Cluster Labels'] == 3, lon_ny_merged.columns[[0]+[1] + list(range(lon_ny_merged.shape[1]-10, lon_ny_merged.shape[1]))]]

In [61]:
print(Cluster3.shape)
Cluster3

(2, 12)


,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,London,Redbridge,Bus Stop,Coffee Shop,Grocery Store,Hotel,Park,Chinese Restaurant,Theater,Art Gallery,Asian Restaurant,Gay Bar
28,London,Tower Hamlets,Bus Stop,Sandwich Place,Grocery Store,Italian Restaurant,Park,Sushi Restaurant,Tennis Court,Bar,Hotel,Gym / Fitness Center


In [62]:
Cluster4=lon_ny_merged.loc[lon_ny_merged['Cluster Labels'] == 4, lon_ny_merged.columns[[0]+[1]+ list(range(lon_ny_merged.shape[1]-10, lon_ny_merged.shape[1]))]]

In [63]:
print(Cluster4.shape)
Cluster4

(12, 12)


,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
101,Manhattan,Washington Heights,Coffee Shop,Bakery,Grocery Store,Mobile Phone Shop,Gym,Liquor Store,Pizza Place,Sandwich Place,Deli / Bodega,Bar
102,Manhattan,Inwood,Coffee Shop,Mexican Restaurant,Restaurant,Bar,Lounge,Bakery,Pizza Place,Deli / Bodega,Park,Chinese Restaurant
103,Manhattan,Hamilton Heights,Coffee Shop,Pizza Place,Bar,Deli / Bodega,Mexican Restaurant,Bakery,Indian Restaurant,Sandwich Place,Caribbean Restaurant,School
104,Manhattan,Manhattanville,Coffee Shop,Sushi Restaurant,Bar,Deli / Bodega,Mexican Restaurant,Italian Restaurant,Liquor Store,Chinese Restaurant,Climbing Gym,Seafood Restaurant
106,Manhattan,East Harlem,Mexican Restaurant,Thai Restaurant,Bakery,Sandwich Place,Latin American Restaurant,Bar,Deli / Bodega,Park,Spa,Grocery Store
110,Manhattan,Roosevelt Island,Park,Coffee Shop,Restaurant,Moving Target,Bubble Tea Shop,Gym / Fitness Center,Gym,Dry Cleaner,Outdoors & Recreation,Soccer Field
124,Manhattan,Manhattan Valley,Coffee Shop,Bar,Mexican Restaurant,Pizza Place,Vietnamese Restaurant,Yoga Studio,Hostel,Bubble Tea Shop,Hill,Noodle House
125,Manhattan,Morningside Heights,Coffee Shop,Bookstore,American Restaurant,Park,Deli / Bodega,Burger Joint,Supermarket,Bar,Sandwich Place,Indian Restaurant
274,Manhattan,Tudor City,Coffee Shop,Mexican Restaurant,Park,Sushi Restaurant,Pizza Place,Greek Restaurant,Dog Run,Deli / Bodega,Bar,Garden
5,London,Camden,Coffee Shop,Greek Restaurant,Pizza Place,Bus Stop,Grocery Store,Chinese Restaurant,French Restaurant,Health & Beauty Service,Shoe Store,Movie Theater


#### <span style="color:gray"> Displaying each cluster on a Folium map </span>

In [64]:
# checking for NaN in the dataframe before mapping
print(lon_ny_merged.shape)
lon_ny_merged.dropna(inplace=True)
print(lon_ny_merged.shape)

(73, 15)
(64, 15)


In [65]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]



# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lon_ny_merged['Latitude'], lon_ny_merged['Longitude'], lon_ny_merged['Neighborhood'], lon_ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [66]:
# !pip install -U numpy

# !pip install -U pandas

# !pip install -U scipy

# !pip install -U scikit-learn

# !pip install -U imbalanced-learn